# Forecast Skill

Validation of operational systems usually includes skill assessment. This Notebook outlines the procedure using `pyposeidon`. 


- Requires:

    - The data folder. See [README](README.md) in this folder.

    - The output of the [FORECAST](./FORECAST.ipynb) Notebook.

In [ ]:
#to use the full width of the browser window
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.6
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.model as pm

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

### Graphics

In [ ]:
import pyposeidon.utils.pplot # initialize matplolib accessor

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
import pyposeidon.utils.hplot # initialize pyviz accessor

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env `PATH`. This is not needed when we are running from python.

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

In [ ]:
import numpy
cpath = numpy.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

## Setup

We will use the output of the [FORECAST](./FORECAST.ipynb) Notebook. Make sure you have executed that before.


The runs are all in different folders and we want to consolidate the station time series.

* Note that because of the overlaping time frames we have a number of forecasted values per location, based on the frequence of the forecasts.

We define the folders involved

In [ ]:
import glob

In [ ]:
folders = glob.glob('./test/forecast/*')
folders.sort()
folders

We combine all available data with

In [ ]:
from pyposeidon.utils.post import to_thalassa

In [ ]:
to_thalassa(folders,freq=6, rpath='./test/thalassa/')

In [ ]:
import xarray as xr

In [ ]:
vdata = xr.open_dataset('./test/thalassa/fskill.nc')

In [ ]:
vdata

In [ ]:
#select node
node = vdata.sel(node=1)

In [ ]:
plt.figure()
node.elevation.dropna(dim='time').plot(color='r',label='model')
node.observation.dropna(dim='time').plot(label='obsrv')
plt.legend()

## Estimate forecasting skill

In [ ]:
#Visualize all forecasts plots
plt.figure()
vdata.isel(node=1).forecast.plot.line(x='ftime')

Using the dataset (vdata), the indicator (RMSE) and the node (1) we can get the relative skill for 12,24,36,48,60,72 forecast lead time

In [ ]:
from pyposeidon.utils.post import fskill

In [ ]:
rr = fskill(vdata,'RMSE',1)
rr